In [1]:
# this note book is used for pairs comparsion for users

In [ ]:
import psycopg2

postgres_conn_params = {
    'host' : '192.168.0.148',
    'port' : '5432',
    'user' : 'postgres',
    'password' : 'hide',
    'database' : 'steam'
}

conn = psycopg2.connect(**postgres_conn_params)

In [4]:
# only select users who had made 10+ reviews totally
SQL = """
    SELECT DISTINCT reviewer_name
    FROM (SELECT reviewer_name
        FROM game_reviews
        WHERE length(review_text) >= 20
        GROUP BY reviewer_name
        HAVING count(game_name) > 10) as temp
"""

cursor = conn.cursor()
cursor.execute(SQL)
users = cursor.fetchall()
cursor.close()

In [5]:
import random
users = [u for (u,) in users]
random.shuffle(users)
print(len(users))
print(users[:5])

147402
['https://steamcommunity.com/profiles/76561197971248094/', 'https://steamcommunity.com/profiles/76561198038954363/', 'https://steamcommunity.com/id/BoneHeadBrianTV/', 'https://steamcommunity.com/profiles/76561198006265186/', 'https://steamcommunity.com/profiles/76561198069439251/']


In [6]:
# save and load, of the shuffled lists since may not able to process all data at a time.
import pickle
with open("data/user_list.pickle", "wb") as f:
    pickle.dump(users, f)

In [ ]:
# Start from here if rerunning the notebook

In [1]:
import pickle
with open("data/user_list.pickle", "rb") as f:
    users = pickle.load(f)

In [2]:
print(users[:5])
print(len(users))

['https://steamcommunity.com/profiles/76561197971248094/', 'https://steamcommunity.com/profiles/76561198038954363/', 'https://steamcommunity.com/id/BoneHeadBrianTV/', 'https://steamcommunity.com/profiles/76561198006265186/', 'https://steamcommunity.com/profiles/76561198069439251/']
147402


In [3]:
# load language model
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ll3_fitting/epoch_329",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
tokenizer.pad_token = '<|reserved_special_token_250|>'
tokenizer.pad_token_id = 128255

In [5]:
def prompt_generation(inputs):
    outputs = []
    for i in range(len(inputs)):
        temp = []
        temp.append({'role': 'system', 'content': "You are a personal judge of video game, your role is to judge which game is preferred by the user based on user's feedbacks of two games. Simply reply with prefered game's name, no need for explanation"})
        temp.append({
            'role': 'user',
            'content': f"I played two games {inputs[i][0][0]}, and {inputs[i][1][0]}. After playing, I gave reviews for both games as follow:\n{inputs[i][0][0]}: {inputs[i][0][1]}\n{inputs[i][1][0]}: {inputs[i][1][1]}"
        })
        outputs.append(temp)
    return outputs

In [6]:
import re
def normalize_title(title: str):
    title = title.replace('.', '')
    title = title.replace(',', '')
    title = title.replace('/', '')
    title = title.replace(' ', '')
    title = title.replace("'", '')
    title = title.replace("-", '')
    # Remove non-ascii characters
    title = title.encode("ascii", errors="ignore").decode()
    # Remove extra spaces
    title = re.sub(' +', ' ', title)
    # Convert to lowercase
    title = title.lower()
    # Strip leading and trailing spaces
    title = title.strip()
    return title

In [ ]:
import psycopg2

postgres_conn_params = {
    'host' : '192.168.0.148',
    'port' : '5432',
    'user' : 'postgres',
    'password' : 'hide',
    'database' : 'steam'
}

conn = psycopg2.connect(**postgres_conn_params)

In [10]:
index = users.index('https://steamcommunity.com/id/vggandros/')
index

9999

In [9]:
import torch
conn.rollback()
queryer = conn.cursor()
saver = conn.cursor()

Insert_SQL = """
                INSERT INTO llama3_pair(reviewer_name, gamea_name, gameb_name, model, prefered_game)
                VALUES (%s, %s, %s, %s, %s)
            """

for i in range(9769, 10000, 1):
    user = users[i]
    SQL = f"""
        SELECT game_name, review_text
        FROM game_reviews
        WHERE reviewer_name = '{user}'
    """
    queryer.execute(SQL)
    reviews = queryer.fetchall()

    processing_queue = []
    for j in range(len(reviews)-1):
        for k in range(j+1, len(reviews)):
            processing_queue.append((reviews[j], reviews[k]))
    
    queue = prompt_generation(processing_queue)
    
    for j in range(0, len(processing_queue), 8): # batch size of 8        
        msgs = queue[j:j+8]
        inputs = tokenizer.apply_chat_template(
            msgs,
            tokenize = True,
            padding = True,
            truncation = False,
            add_generation_prompt = True, # Must add for generation
            return_tensors = "pt",
        )

        
        if(inputs.size(dim=1) > 1020): # if this batch contains very long review text, reduce the batch size to 2
            for jj in range(0, len(msgs), 2):
                msgs = queue[j+jj:j+jj+2]
                inputs = tokenizer.apply_chat_template(
                    msgs,
                    tokenize = True,
                    padding = True,
                    truncation = False,
                    add_generation_prompt = True, # Must add for generation
                    return_tensors = "pt",
                ).to('cuda')
                if(inputs.size(dim=1) >= 4090): # exceed the limit of llama3
                    continue
                    
                attention_mask = (inputs != tokenizer.pad_token_id).int()
    
                outputs = model.generate(input_ids = inputs, max_new_tokens = 4096, attention_mask = attention_mask, use_cache = True)
                gen_idx = len(inputs[0])
                result = tokenizer.batch_decode(outputs[:, gen_idx:], skip_special_tokens = True)

                for k in range(len(result)):
                    if normalize_title(result[k]) == normalize_title(processing_queue[j+jj+k][0][0]):
                        prefered_game = processing_queue[j+jj+k][0][0]
                    elif normalize_title(result[k]) == normalize_title(processing_queue[j+jj+k][1][0]):
                        prefered_game = processing_queue[j+jj+k][1][0]
                    else:
                        prefered_game = "CannotJudge"
        
                    saver.execute(Insert_SQL, (user, processing_queue[j+jj+k][0][0], processing_queue[j+jj+k][1][0], "unsloth_e329", prefered_game))
                conn.commit()
            continue

        inputs.to('cuda')
        
        attention_mask = (inputs != tokenizer.pad_token_id).int()

        outputs = model.generate(input_ids = inputs, max_new_tokens = 4096, attention_mask = attention_mask, use_cache = True)
        gen_idx = len(inputs[0])
        result = tokenizer.batch_decode(outputs[:, gen_idx:], skip_special_tokens = True)

        for k in range(len(result)):
            if normalize_title(result[k]) == normalize_title(processing_queue[j+k][0][0]):
                prefered_game = processing_queue[j+k][0][0]
            elif normalize_title(result[k]) == normalize_title(processing_queue[j+k][1][0]):
                prefered_game = processing_queue[j+k][1][0]
            else:
                prefered_game = "CannotJudge"

            saver.execute(Insert_SQL, (user, processing_queue[j+k][0][0], processing_queue[j+k][1][0], "unsloth_e329", prefered_game))
        conn.commit()

    if i%25 == 0:
        print(f"processed {i} users, {i}th user is {user}")
        torch.cuda.empty_cache() # clear cache every 25 users

processed 9775 users, 9775th user is https://steamcommunity.com/id/SwampLord420/
processed 9800 users, 9800th user is https://steamcommunity.com/id/lil_hj/
processed 9825 users, 9825th user is https://steamcommunity.com/id/ViolenceFaith/
processed 9850 users, 9850th user is https://steamcommunity.com/profiles/76561198249595398/
processed 9875 users, 9875th user is https://steamcommunity.com/id/iSshkin/
processed 9900 users, 9900th user is https://steamcommunity.com/id/GrandeLucao/
processed 9925 users, 9925th user is https://steamcommunity.com/profiles/76561198079437668/
processed 9950 users, 9950th user is https://steamcommunity.com/id/latsyrcmoy/
processed 9975 users, 9975th user is https://steamcommunity.com/id/braprancher/
